## Load extensions and run imports

In [ ]:
%load_ext nb_black
import pandas as pd

## Load challenge report data

In [ ]:
df = pd.read_csv('SuccessfulChallengeReport.csv', engine='pyarrow')

### EDA Code

Useful for checking loaded data.

In [ ]:
# df.info()
# df.NullAddressAllowed.value_counts()

## Get reference data

Load BSL lookup data.

In [ ]:
locLookup = pd.read_parquet('bslsLookup.parquet')

Load challenge type to reason data

In [ ]:
reasons = pd.read_csv('reasons.csv')

Load state usps and names to FIPS codes.

In [ ]:
stateColumnHints = {"stateFips":str}
states = pd.read_csv('states.csv', dtype=stateColumnHints)

### EDA Code

Useful for checking loaded data.

In [ ]:
states.info()
locLookup.info()
df.info()
states

## Merge location data 

In [ ]:
merged = df.merge(locLookup, how='left', on='location_id')

### EDA Code

In [ ]:
merged.info()

## Enrich Data

Extract state FIPS code from block_geoid.

In [ ]:
merged['stateFips'] = merged.block_geoid.str[0:2]

Merge in reason codes.

In [ ]:
merged = merged.merge(reasons, how='left', on='ChallengeType')

Merge in state names based on FIPS codes

In [ ]:
merged = merged.merge(states, how='left', on='stateFips')

## Output Data

Output data into Excel. Then open the file and create a combined state column using the following formula:

* Create a table of the data
* Resize columns
* Add a combinedState Column
* Use the formula below to create combined column and preference the state provided in the original data and if it is blank then uses the computed state from the lookup table
```excel
=IF(ISBLANK([@State]),[@usps],[@State])
```

In [ ]:
merged.to_excel('mergedSuccessfulChallengeReport.xlsx', index=False)